In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("tanzania.csv")
df.head()

,GA,FHR_during_labour,Amniotic_fluid,Delivery_Mode,CS_category,CS_indication,BW,Apgar_1min,Apgar_5min,BMV,Outcome_30min,Outcome_24hours,Outcome_7days,Time_of_death,temp12ad0,puls13ad0,oxy14ad0,HIE_highest,major_cause,outcome
0,42.0,1,2,1,NaN,NaN,3000,8,10,2,2,2,3.0,2.0,36.0,144.0,60.0,NaN,6.0,1
1,40.0,9,1,1,NaN,NaN,2500,9,10,2,2,3,NaN,1.0,35.4,155.0,50.0,NaN,6.0,1
2,38.0,1,1,1,NaN,NaN,2815,7,10,1,1,2,3.0,2.0,36.4,166.0,71.0,NaN,6.0,1
3,36.0,1,1,1,NaN,NaN,2125,8,7,1,2,2,3.0,2.0,34.9,136.0,71.0,NaN,6.0,1
4,32.0,9,1,1,NaN,NaN,2115,9,10,2,2,2,3.0,3.0,36.1,131.0,72.0,NaN,6.0,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 20 columns):
GA                   599 non-null float64
FHR_during_labour    671 non-null int64
Amniotic_fluid       671 non-null int64
Delivery_Mode        671 non-null int64
CS_category          262 non-null float64
CS_indication        261 non-null float64
BW                   671 non-null int64
Apgar_1min           671 non-null int64
Apgar_5min           671 non-null int64
BMV                  671 non-null int64
Outcome_30min        671 non-null int64
Outcome_24hours      671 non-null int64
Outcome_7days        410 non-null float64
Time_of_death        124 non-null float64
temp12ad0            377 non-null float64
puls13ad0            363 non-null float64
oxy14ad0             371 non-null float64
HIE_highest          154 non-null float64
major_cause          124 non-null float64
outcome              671 non-null int64
dtypes: float64(10), int64(10)
memory usage: 104.9 KB


## Data Cleaning

In [5]:
target = 'outcome'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('#NULL!',np.nan,regex=True)
#df = df.replace('99',np.nan,regex=True)
#df = df.replace(99,np.nan,regex=True)
#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = []
df = df.drop(outliers,axis=1)
df.head()

,GA,FHR_during_labour,Amniotic_fluid,Delivery_Mode,CS_category,CS_indication,BW,Apgar_1min,Apgar_5min,BMV,Outcome_30min,Outcome_24hours,Outcome_7days,Time_of_death,temp12ad0,puls13ad0,oxy14ad0,HIE_highest,major_cause,outcome
0,42.0,1,2,1,NaN,NaN,3000,8,10,2,2,2,3.0,2.0,36.0,144.0,60.0,NaN,6.0,1
1,40.0,9,1,1,NaN,NaN,2500,9,10,2,2,3,NaN,1.0,35.4,155.0,50.0,NaN,6.0,1
2,38.0,1,1,1,NaN,NaN,2815,7,10,1,1,2,3.0,2.0,36.4,166.0,71.0,NaN,6.0,1
3,36.0,1,1,1,NaN,NaN,2125,8,7,1,2,2,3.0,2.0,34.9,136.0,71.0,NaN,6.0,1
4,32.0,9,1,1,NaN,NaN,2115,9,10,2,2,2,3.0,3.0,36.1,131.0,72.0,NaN,6.0,1


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,50)
df.info()

number of bad columns: 10
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 10 columns):
FHR_during_labour    671 non-null int64
Amniotic_fluid       671 non-null int64
Delivery_Mode        671 non-null int64
BW                   671 non-null int64
Apgar_1min           671 non-null int64
Apgar_5min           671 non-null int64
BMV                  671 non-null int64
Outcome_30min        671 non-null int64
Outcome_24hours      671 non-null int64
outcome              671 non-null int64
dtypes: int64(10)
memory usage: 52.5 KB


### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,4)
df.info()

number of bad rows: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 10 columns):
FHR_during_labour    671 non-null int64
Amniotic_fluid       671 non-null int64
Delivery_Mode        671 non-null int64
BW                   671 non-null int64
Apgar_1min           671 non-null int64
Apgar_5min           671 non-null int64
BMV                  671 non-null int64
Outcome_30min        671 non-null int64
Outcome_24hours      671 non-null int64
outcome              671 non-null int64
dtypes: int64(10)
memory usage: 52.5 KB


In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 0


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  4   4   5 302   9  12   2   2   3]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [6, 7]
i_category: [0, 1, 8]
variable type: [2. 2. 3. 3. 3. 3. 1. 1. 2.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 671 entries, 0 to 670
Data columns (total 9 columns):
FHR_during_labour    671 non-null int64
Amniotic_fluid       671 non-null int64
Delivery_Mode        671 non-null int64
BW                   671 non-null int64
Apgar_1min           671 non-null int64
Apgar_5min           671 non-null int64
BMV                  671 non-null int64
Outcome_30min        671 non-null int64
Outcome_24hours      671 non-null int64
dtypes: int64(9)
memory usage: 52.4 KB


## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(671, 17)
[[1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([547, 124]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')